<a href="https://colab.research.google.com/github/SushmitalKhan/Dissertation/blob/main/infer_condition2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
This condition will not ask users what type of inference they want. It will generate inferences, and rate the uncommonness and sensitivity of the inferences.

In [15]:
import openai

openai.api_key = ''

# tokenizer = AutoTokenizer.from_pretrained("gpt2")  # GPT-3.5 & GPT-4 use GPT-2 tokenizer

def count_tokens(text):
    """Returns the number of tokens in a given text."""
    return len(tokenizer.encode(text))

def truncate_list(data_list, max_tokens):
    """Truncates a list to fit within a token budget."""
    truncated_list = []
    token_count = 0
    for item in data_list:
        item_tokens = count_tokens(str(item))  # Count tokens in each item
        if token_count + item_tokens <= max_tokens:
            truncated_list.append(item)
            token_count += item_tokens
        else:
            break
    return truncated_list

def run_prompt(prompt):
    response = openai.chat.completions.create(
        # model="gpt-3.5-turbo",  # Changed 'engine' to 'model'
        model="gpt-4o-mini",
        # model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1000,
        # max_tokens=min(1000, 16385 - input_token_count)  # Ensure within token limit
        temperature=0.7,
        n=1, # Return only one response
        stop=None # optional stopping sequence
        # request_timeout = 60
        )
    json_response = response.choices[0].message.content.strip()

    return json_response  # Convert response to Python dictionary

In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount('/content/drive')
import json
import datetime
from pathlib import Path

In [55]:
powerset2023 = pd.read_csv('/content/drive/MyDrive/Dissertation/Study 1/Powerset/powerset_by_year/allActivity_2023.csv')

In [56]:
# Select columns with less than 600 non-NaN rows
misc_columns = powerset2023.columns[powerset2023.notna().sum() < 600]

# Combine values into a new column
powerset2023['takeout1_misc_MyActivity_Search Title'] = powerset2023[misc_columns].apply(
    lambda row: ', '.join(row.dropna().astype(str)),
    axis=1)

# Drop the identified misc_columns
powerset2023 = powerset2023.drop(columns=misc_columns)


# Drop duplicate values in the specified column, keeping the first occurrence
powerset2023 = powerset2023.drop_duplicates(subset=['takeout1_chrome_MyActivity_Search Title'], keep='first')

powerset2023 = powerset2023.rename(columns={
    "old_header2": "new_name2",
    "takeout1_imageSearch_MyActivity_Search Title" : "Image_Search",
    "takeout1_chrome_MyActivity_Search Title" : "Browser_history",
    "takeout1_maps_MyActivity_Search Title" : "Location_history",
    "takeout1_YT_search-history_Search Title" : "YT_search_history",
    "takeout1_YT_watch-history_Search Title" : "YT_watch_history",
    "takeout1_misc_MyActivity_Search Title" : "Misc"
    })

make prompt more dynamic --> ask for user input for inference and recommendation #

*prompt: recommend one product based on these three inferences*

make numeric instead of text --> parameterize the model --> multiply it out --> if you recommend one product based on the inferences; for example inference 1 recommendation 1, inference 1 recommendation 2 and so on

In [ ]:
#Ask to select any four column
print("\nPlease select four datasets from the options (datasets should not be the same):")
for i, col in enumerate(powerset2023.columns, 1):
    print(f"{i}. {col}")

# Ask the user to select any 4 columns
selected_columns = []
df_selected = pd.DataFrame()
while len(selected_columns) < 4:
    try:
        col_num = int(input(f"\nSelect column {len(selected_columns) + 1} by number (1-{len(powerset2023.columns)}): "))
        if 1 <= col_num <= len(powerset2023.columns):
            col_name = powerset2023.columns[col_num - 1]
            if col_name not in selected_columns:
                selected_columns.append(col_name)
            else:
                print("You've already selected this column. Try another.")
        else:
            print("Invalid number. Please choose a number from the list.")
    except ValueError:
        print("Invalid input. Please enter a number.")

# Extract the selected columns
powerSubset2023 = powerset2023[selected_columns]

# Print the selected DataFrame
print("\nSelected DataFrame:")

In [ ]:
# # Optionally, save to a CSV file
save_option = input("\nWould you like to save this selection to a CSV file? (yes/no): ").strip().lower()
if save_option == "yes":
    df_selected.to_csv("/content/drive/MyDrive/Dissertation/Study 1/Inference_data/24APRIL2025/chrome_maps_YTwatch_misc.csv", index=False)
    # print("File saved as 'selected_input_data.csv'.")


Would you like to save this selection to a CSV file? (yes/no): no


In [ ]:
#Ask to select how many inferences they wat

# Define the options
options = [3,6,9,12]

# Display the options
print("Please select a number from the options below by typing the corresponding number:\n")
for i, option in enumerate(options, 1):
    print(f"{i}. {option}")

# Get user input
user_input = input("\nEnter your selection (1-4): ")

# Validate input
if user_input.isdigit() and 1 <= int(user_input) <= 4:
    inference_no = options[int(user_input) - 1]
    print(f"\nYou selected: {inference_no}")
else:
    print("\nInvalid selection. Please enter a number between 1 and 4.")

# inference_input = int(inference_input)

In [59]:
def clean_json_string(json_string):
  """Cleans a JSON string by removing unwanted characters and standardizing spacing."""

  # cleaned = re.sub(r"```json ", "", json_string)
  # Replace extra spaces and newlines with single spaces
  cleaned = re.sub(r"```json|```", " ", json_string)
  # Remove characters outside the basic multilingual plane
  cleaned = re.sub(r"[^\u0000-\uFFFF]", "", cleaned)
  # Try to parse the cleaned JSON; return original if invalid
  try:
    return json.loads(cleaned)
  except json.JSONDecodeError:
    print(f"Warning: Failed to parse JSON, returning original: {json_string[:50]}...")
    return cleaned

# Apply the function to each item in your list

In [114]:
import itertools
from pathlib import Path
from datetime import datetime
import json

def build_prompts_for_user_combinations(
    powerSubset2023,        # DataFrame with user-selected columns
    inference_no,           # number of inferences per prompt
    rows,                   # number of rows to sample from each dataset
    combo_sizes,            # int or list/tuple of combination sizes
    folder="outputs"        # folder to save JSON
):
    """
    Build prompts for all possible combinations of columns in the DataFrame
    for the specified combination sizes.
    """

    # If user passes a single integer, convert to a list
    if isinstance(combo_sizes, int):
        combo_sizes = [combo_sizes]

    col_names = list(powerSubset2023.columns)
    inference_json_list = []

    for combo_size in combo_sizes:
        # skip invalid sizes
        if combo_size < 1 or combo_size > len(col_names):
            print(f"⚠️ Skipping invalid combination size: {combo_size}")
            continue

        # generate all combinations of the specified size
        all_combos = list(itertools.combinations(col_names, combo_size))

        for selected_cols in all_combos:
            columns_str = " AND ".join(selected_cols)

            # fetch data for each column
            col_data_list = [powerSubset2023[:rows][col] for col in selected_cols]
            col_data_str = " AND ".join(
                f"{col_data.dropna().tolist()} in {col_name}"
                for col_data, col_name in zip(col_data_list, selected_cols)
            )

            prompt = f"""
            User wants to know what {col_data_str} tells about them.
            1. {inference_no} inferences about the user based on their online behavior in the form "interested in X".
            2. Rate uncommonness (1–10) and sensitivity (1–10) for each inference.
            3. Recommend **EXACTLY ONE** product for each group.
            4. Recommend **EXACTLY ONE** product based on all {inference_no} inferences for {columns_str}.

            STRICT JSON OUTPUT ONLY.
            """

            print(f"🔹 Sending prompt to GPT for columns: {columns_str}...")
            assistant_reply = run_prompt(prompt)

            inference_json_list.append({
                "combined_cols": selected_cols,
                "combo_size": combo_size,
                "gpt_output": assistant_reply
            })

    # Save JSON
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"inferences_user_combinations_{timestamp}.json"
    filepath = Path(folder) / filename
    filepath.parent.mkdir(parents=True, exist_ok=True)

    return inference_json_list, filepath


In [ ]:
inference_json, output_file = build_prompts_for_user_combinations(
    powerSubset2023,
    inference_no= inference_no,
    rows= 200, #Update number of rows to the full dataset
    combo_sizes= [1,2,3,4],
    folder="/content/drive/MyDrive/Dissertation/Study 1/Inference_data/"
)

In [122]:
def write_json(output_filename, inference_json):
    if not inference_json:
        print("⚠️ No data to write. JSON file not created.")
        return None

    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(inference_json, f, indent=4, ensure_ascii=False)
    print(f"✅ JSON saved successfully as {output_filename}")
    return output_filename

In [123]:
# Apply clean_json_string to GPT outputs only
for entry in inference_json:
    if isinstance(entry.get("gpt_output"), str):
        entry["gpt_output"] = clean_json_string(entry["gpt_output"])

In [124]:
write_json(
    output_file, inference_json
)

✅ JSON saved successfully as /content/drive/MyDrive/Dissertation/Study 1/Inference_data/inferences_user_combinations_20250911_222951.json


PosixPath('/content/drive/MyDrive/Dissertation/Study 1/Inference_data/inferences_user_combinations_20250911_222951.json')

Ask LLM while generating or retrospectively to rate each of these inferences as a level of uncommonness. Add a column with inference score of 1 to 10 for uncommonness. Higher score to uncommon inferences than some of the others.

Pick top N of the uncommon (unexpected), sensitive ones (including combinations) and use them

Robustness in repitition and generalizabiliy